In [4]:
import pprint as pp
import pandas as pd
import os
import folium
import folium.plugins
import geopandas as gpd
from shapely.geometry import Point


ModuleNotFoundError: No module named 'Point'

In [ ]:
LA_data = pd.read_csv('analysis_data/LA_data.csv')

gdf = gpd.read_file(r"data\network['Los Angeles , California, USA'].gpkg", layer='nodes')


In [ ]:
# Create the geometry column using the Start_Lat and Start_Lng columns
geometry = [Point(xy) for xy in zip(LA_data['Start_Lng'], LA_data['Start_Lat'])]

# Create the GeoDataFrame
LA_gdf = gpd.GeoDataFrame(LA_data, geometry=geometry)


# Create projection accurate to Los Angeles
LA_gdf.set_crs(epsg=4326, inplace=True)
LA_gdf = LA_gdf.to_crs(epsg=6423)
gdf.to_crs(epsg=6423, inplace=True)





NameError: name 'LA_data' is not defined

In [ ]:

def add_accident_counts(gdf, LA_gdf, range_):
    gdf = gdf.copy()
    LA_gdf = LA_gdf.copy()
    gdf['geometry'] = gdf.geometry.buffer(range_)
    joined = gpd.sjoin(LA_gdf, gdf, how='inner', op='intersects')
    
    def nearest_intersection(accident_ID):
        intersections = joined[joined['ID'] == accident_ID].index_right
        if not intersections.empty:
            nearest = min(intersections, key=lambda intersection: LA_gdf[LA_gdf['ID'] == accident_ID].geometry.distance(gdf.loc[intersection].geometry).min())
            return nearest
        else:
            return None

    LA_gdf['nearest_intersection'] = LA_gdf['ID'].apply(nearest_intersection)
    accident_counts = LA_gdf['nearest_intersection'].value_counts()
    gdf['accident_count'] = gdf.index.map(accident_counts.to_dict())
    gdf['accident_count'] = gdf['accident_count'].fillna(0)
    gdf.to_file(f'analysis_data/intersections_with_accidents_range{range_}.gpkg', driver='GPKG')

range_list = [5]

for i in range_list:
    add_accident_counts(gdf, LA_gdf, i)
    print(f'Range {i} meters done')

